# Import Modules
This section imports the required modules

In [1]:
import pandas as pd
from zipfile import ZipFile

In [18]:
zip_path = './data/human-trafficking-victims-dataset-ctdc.zip'
data_file = None
raw_data = None
with ZipFile(zip_path) as zip_file:
    for commpresed in zip_file.filelist:
        if commpresed.filename.endswith('csv'):
            with zip_file.open(commpresed) as data_file:
                raw_data = pd.read_csv(
                    data_file,
                    sep=';',
                    header=0,
                    low_memory=False,
                    usecols=range(1,64)
                )

In [21]:
raw_data.shape

(97750, 63)

In [23]:
raw_data.tail()

,yearOfRegistration,Datasource,gender,ageBroad,majorityStatus,majorityStatusAtExploit,majorityEntry,citizenship,meansOfControlDebtBondage,meansOfControlTakesEarnings,...,typeOfSexPrivateSexualServices,typeOfSexConcatenated,isAbduction,RecruiterRelationship,CountryOfExploitation,recruiterRelationIntimatePartner,recruiterRelationFriend,recruiterRelationFamily,recruiterRelationOther,recruiterRelationUnknown
97745,NaN,Case Management,Male,9--17,Minor,NaN,NaN,CM,NaN,NaN,...,NaN,NaN,NaN,NaN,LY,0.0,0.0,0.0,0.0,1.0
97746,NaN,Case Management,Male,9--17,Minor,NaN,NaN,CM,NaN,NaN,...,NaN,NaN,NaN,NaN,LY,0.0,0.0,0.0,0.0,1.0
97747,NaN,Case Management,Male,9--17,Minor,NaN,NaN,CM,NaN,NaN,...,NaN,NaN,NaN,NaN,LY,0.0,0.0,0.0,0.0,1.0
97748,NaN,Case Management,Male,9--17,Minor,NaN,NaN,CM,NaN,NaN,...,NaN,NaN,NaN,NaN,LY,0.0,0.0,0.0,0.0,1.0
97749,NaN,Case Management,Male,9--17,Minor,NaN,NaN,CM,NaN,NaN,...,NaN,NaN,NaN,NaN,LY,0.0,0.0,0.0,0.0,1.0


In [ ]:

print("hola mundo ")